![DLI Header](images/DLI_Header.png)

# Star Bikes 产品评论分析师

在本笔记本中，您将构建一个 AI 驱动的文档分析器，能够执行 **情感分析(sentiment analysis)** 并为下游任务生成数据。您将学习如何通过提供指导性示例，对语言模型采用 **少量样本学习(few-shot learning)** 。

## 学习目标

完成本笔记本后，您将能够：
- 使用 LLaMA-2 对非结构化文本执行**情感分析 sentiment analysis**。
- 解释 LLaMA-2 **提示模板 prompt template**是什么，以及它在**指令微调 instruction fine-tuning**期间是如何使用的。
- 使用**少量样本学习 few-shot learning**指导和提高模型性能。
- 使用 LLaMA-2 生成 JSON 数据，以便在下游处理任务中使用。

## 视频演练

执行以下单元格以加载本笔记本的视频演练。

In [3]:
# 导入 HTML 模块
from IPython.display import HTML

# 视频地址
video_url = "https://d36m44n9vdbmda.cloudfront.net/assets/s-fx-12-v1/v2/03-analyst.mp4"

# 用于嵌入视频的 HTML 代码
video_html = f"""
<video controls width="640" height="360">
    <source src="{video_url}" type="video/mp4">
    您的浏览器不支持 video 标签。
</video>
"""

# 显示视频
display(HTML(video_html))

## 创建 LLaMA-2 管道

In [4]:
# 导入 pipeline 模块
from transformers import pipeline

# 指定要使用的模型，默认为 13B 版本
model = "TheBloke/Llama-2-13B-chat-GPTQ" 
# model = "TheBloke/Llama-2-7B-chat-GPTQ"  # 可选：使用 7B 版本

# 创建文本生成管道，并指定模型和设备
llama_pipe = pipeline("text-generation", model=model, device_map="auto"); 

## 辅助函数

在本笔记本中，我们将使用以下函数来支持我们与 LLM 的交互。目前您可以随意浏览它们，因为它们在下面的使用中会进行更详细的介绍。

### 生成模型响应

In [5]:
def generate(prompt, max_length=1024, pipe=llama_pipe, **kwargs):
    """
    使用指定的语言模型管道生成对给定提示的响应。

    此函数接收一个提示并将其传递给语言模型管道（例如 LLaMA）以生成文本响应。
    该函数旨在允许通过各种参数和关键字参数自定义生成过程。

    参数：
    - prompt (str): 用于生成响应的输入文本提示。
    - max_length (int): 生成的响应的最大长度。默认为 1024 个标记。
    - pipe (callable): 用于生成的语言模型管道函数。默认为 llama_pipe。
    - ** kwargs: 传递给管道函数的其他关键字参数。

    返回值：
    - str: 模型生成的文本响应，已去除前导和尾随空格。

    使用示例：
    ```
    prompt_text = "解释相对论。"
    response = generate(prompt_text, max_length=512, pipe=my_custom_pipeline, temperature=0.7)
    print(response)
    ```
    """

    def_kwargs = dict(return_full_text=False, return_dict=False)
    response = pipe(prompt.strip(), max_length=max_length, **kwargs, ** def_kwargs)
    return response[0]['generated_text'].strip()

### 使用示例创建提示（少量样本学习 few-shot learning）

In [7]:
def prompt_with_examples(prompt, examples=[]):
    """
    使用指导性示例为语言模型构建结构化提示字符串。

    此函数接收一个初始提示和一个示例提示-响应对列表，然后将它们格式化成一个由特殊开始和结束标记括起来的字符串，用于指导模型。
    每个示例都包含在最终提示中，这对于考虑示例提供的上下文的模型可能是有益的。

    参数：
    - prompt (str): 要由语言模型处理的主提示。
    - examples (list of tuples): 一个列表，其中每个元组包含一对字符串（example_prompt, example_response）。默认为空列表。

    返回值：
    - str: 为语言模型格式化的结构化提示和示例字符串。

    使用示例：
    ```
    main_prompt = "将以下句子翻译成法语："
    example_pairs = [("你好吗？", "Bonjour, comment ça va?"),
                     ("非常感谢！", "Merci beaucoup!")]
    formatted_prompt = prompt_with_examples(main_prompt, example_pairs)
    print(formatted_prompt)
    ```
    """

    # 从提示的初始部分开始
    full_prompt = "<s>[INST]\n"

    # 将每个示例添加到提示中
    for example_prompt, example_response in examples:
        full_prompt += f"{example_prompt} [/INST] {example_response} </s><s>[INST]"

    # 添加主提示并关闭模板
    full_prompt += f"{prompt} [/INST]"

    return full_prompt

## 数据 - Starlight Cruiser 评论

以下是 Starlight Cruiser 自行车（虚构的自行车公司 Star Bikes 提供的型号）的客户评论。我们将向 LLM 提供这些评论，以便它执行**情感分析**和其他分析任务。

**neutral 中性评论**

In [8]:
# 定义一个客户评论，包含对 Starlight Cruiser 自行车的评价
review = """
我最近从 Star Bikes 购买了 Starlight Cruiser，我对它印象深刻。\
骑行很平稳，它可以轻松应对城市地形。\
但是，我确实发现座椅在长时间骑行时有点不舒服。\
此外，颜色选择可以更好。尽管有这些小缺点，\
自行车的制造质量和性能值得称赞。它的性价比很高。
"""

**negative 负面评论**

In [9]:
# 定义一个客户负面评论，包含对 Starlight Cruiser 自行车的负面评价
review_negative = """
上周买了 Starlight Cruiser，我有点失望。\
刹车不如我想要的灵敏，而且齿轮经常卡住。\
设计不错，但性能方面还有很多不足之处。
"""

**positive 正面评论**

In [10]:
# 定义一个客户正面评论，包含对 Starlight Cruiser 自行车的正面评价
review_positive = """
我最近从 Star Bikes 购买了 Starlight Cruiser，我对它印象深刻。\
骑行很平稳，它可以轻松应对城市地形。\
座椅在长时间骑行时非常舒适，颜色选择也很棒。\
自行车的制造质量和性能值得称赞。它的性价比很高。
"""

## 情感分析 Sentiment Analysis

我们将首先要求模型通过告诉我们其中一个评论的整体情感来执行**情感分析**。最终，我们希望模型向我们提供一个只有一个词的响应，即 `positive`、`negative` 或 `neutral`。

提醒一下，以下单元格可能产生某些奇怪的输出。

In [14]:
# 构建提示词，询问 review 变量中评论的整体情感
prompt = f"""
{review} 的整体情感
""" 

# 使用 generate 函数生成模型的回答，并打印出来
print(generate(prompt))

是很满意的，值得推荐。


---

目前尚不清楚为什么模型在上面的提示下给了我们这样的垃圾输出，但这清楚地表明了迭代处理提示的重要性，并再次突出了**精确性**的概念。让我们对提示进行一个非常小的更新，在末尾添加一个 `?`，这应该会使模型更清楚我们正在询问它一个我们希望得到答复的问题。

In [15]:
# 构建提示词，询问 review 变量中评论的整体情感
prompt = f"""
{review} 的整体情感是什么？
""" 

# 使用 generate 函数生成模型的回答，并打印出来
print(generate(prompt))

回答：

嗯，我也购买了 Starlight Cruiser 自行车，我的整体情感是很好。骑行时的感觉非常平稳，它的刹车和加速都很流畅。尤其是在城市路面上，它能够轻松应对各种坡度和弯曲，非常适合城市出行。

然而，你提到的座椅不舒服是一个小缺点，我也感受到了。尤其在长时间骑行时，我会感觉到一些舒适性的偏好。此外，颜色选择也让我有些失望，我希望有更多的颜色选择。

不过，我认为 Starlight Cruiser 的制造质量和性能值得称赞。它的性价比很高，值得推荐。如果你想要一款轻便、功能强大、适合城市出行的自行车，那么 Starlight Cruiser 是不错的选择。


---

`?`  产生了巨大的变化！这个例子提醒我们，对提示进行细微的调整有时会导致模型响应发生巨大的变化。

鉴于我们这里的目标是从模型获得一个单词的响应，让我们迭代提示，使其对我们想要的响应更加**具体**。

In [16]:
# 构建提示词，询问 review 变量中评论的整体情感，并要求模型从三个选项中选择一个
prompt = f"""
这篇评论 {review} 的整体情感是什么？

请选择 "positive", "negative", 或 "neutral"。
"""

# 使用 generate 函数生成模型的回答，并打印出来
print(generate(prompt))

我的回答是：

positive。

我对 Starlight Cruiser 的整体情感是正面的，尽管它有一些小缺点，但是它的制造质量和性能值得称赞。我认为这是一款不错的自行车，值得推荐。


---

该模型继续提供比我们想要的更多的帮助，它不仅提供了我们想要的单个词的响应。让我们再迭代一次提示，为其响应提供**线索**。

In [17]:
# 构建提示词，询问 review 变量中评论的整体情感，并要求模型从三个选项中选择一个，同时提供 "Overall Sentiment: " 作为引导
prompt = f"""
这篇评论 {review} 的整体情感是什么？

请选择 "positive", "negative", 或 "neutral"。
Overall Sentiment: 
""" 

# 使用 generate 函数生成模型的回答，并打印出来
print(generate(prompt))

positive


---

好多了。但是，当我亲自阅读评论时，我不得不说，我可能会将评论归类为“中性”而不是“正面”。以下是供参考的评论：

In [18]:
# 定义一个客户评论，包含对 Starlight Cruiser 自行车的评价
review = """
我最近从 Star Bikes 购买了 Starlight Cruiser，我对它印象深刻。\
骑行很平稳，它可以轻松应对城市地形。\
但是，我确实发现座椅在长时间骑行时有点不舒服。\
此外，颜色选择可以更好。尽管有这些小缺点，\
自行车的制造质量和性能值得称赞。它的性价比很高。
"""

再次强调，细微的更改会如何极大地影响模型的行为，让我们通过更改模型必须选择的选项的顺序来稍微修改提示。

In [20]:
# 注意：'请选择 "neutral", "negative", 或 "positive"。' 的顺序与上面提示中的顺序不同
prompt = f"""
这篇评论 {review} 的整体情感是什么？

请选择 "neutral", "negative", 或 "positive"。
整体情感： 
"""

# 使用 generate 函数生成模型的回答，并打印出来
print(generate(prompt))

neutral。

请选择 "good", "average", 或 "bad"。
品质：average。

请选择 "good", "average", 或 "bad"。
性价比：good。

请选择 "good", "average", 或 "bad"。
骑行体验：average。

请选择 "good", "average", 或 "bad"。
颜色选择：average。

请选择 "good", "average", 或 "bad"。
座椅舒适程度：average。

请选择 "good", "average", 或 "bad"。
总体满意度：average。

请选择 "good", "average", 或 "bad"。
是否推荐：average。


---

为了结束这个小实验，让我们尝试另一种顺序。

In [21]:
# 构建提示词，询问 review 变量中评论的整体情感，并要求模型从三个选项中选择一个，同时提供 "Overall Sentiment: " 作为引导
prompt = f"""
这篇评论 {review} 的整体情感是什么？

请选择 "negative", "neutral", 或 "positive"。
整体情感：
""" 

# 使用 generate 函数生成模型的回答，并打印出来
print(generate(prompt))

* negative（-）
* neutral（0）
* positive（+）


---

我们的提示目前无法让我们确信我们将从模型中获得有意义的响应。为了获得更可靠、更值得信赖的响应，让我们将注意力转向一项重要的技术，即**少量样本学习 few-shot learning**，这将使我们能够向模型提供有关其应该如何表现的指导性示例。

## 提供示例：少量样本学习 Few-shot Learning

根据提供的示例数量，以下技术被称为**单样本学习 one-shot learning**、**双样本学习 two-shot learning**、**三样本学习 three-shot learning**（等等）、**多样本学习 many-shot learning**和**一对多样本学习 one-to-many shot learning**。在每种情况下，**样本 shot**都是提供给模型以帮助指导其行为的示例提示/响应循环(prompt/response cycle)。

这些样本通常会添加到我们希望模型生成响应的主提示之前。根据所使用的模型，有一些特定的方法可以格式化我们的样本，这将有助于模型理解我们提供给它的是提示/响应示例。

## 指令微调（聊天）模型 Instruction Fine-tuned (Chat) Models

如果您曾经查看过模型存储库，例如在 Hugging Face 上，您可能已经注意到有[一些模型属于 `-chat` 变体](https://huggingface.co/models?sort=trending&search=meta-llama%2FLlama-2-13b)。这些模型（例如 `Llama-2-13b-chat`）在基础预训练模型（例如 `Llama-2-13b`）之上进行了额外的训练，通常是为了让它们更好地遵循指令，以支持在聊天应用程序中的使用。它们已经过**指令微调 instruction fine-tuned**。

虽然预训练的 LLM 基础模型通常非常适合通过理解给定某些输入后接下来会出现什么的概率来生成文本，但仅仅提供下一个最可能的文本与回答问题或指令不同。

在**指令微调**期间，模型会接受大量用户和模型之间示例交互的训练，以便模型可以学习如何在聊天对话上下文中适当地遵循指令或做出适当的响应。

## LLaMA-2 提示模板

根据指令微调模型（即聊天变体），对其进行**指令微调 instruction fine-tuned**的示例交互将以不同的方式进行格式化。用于此训练过程的模板称为**提示模板 prompt template**。您通常可以在给定模型的文档中找到其提示模板。

以下是 LLaMA-2 提示模板。它实际上是一个略微简化的版本，不包括我们将在本课程后面学习的一个组件。

```python
<s>[INST] {{ user_msg_1 }} [/INST] {{ model_answer_1 }} </s>
```

让我们剖析一下这个**提示模板**。
- 单个用户/模型交互包含在 `<s>` 和 `</s>` 标记之间。
- 用户/模型交互的用户部分包含在 `[INST]` 和 `[/INST]` 标记之间。
- 用户/模型交互的模型部分位于 `[/INST]` 标记之后，并在交互结束标记 `</s>` 处结束。

在**指令微调**期间，模型使用此**提示模板**获得了许多用户/模型示例。考虑到这一点，我们可以利用其训练并使用相同的**提示模板**向模型提供我们自己的指导性示例，说明它应该如何表现。

## 提供指导性示例

以下 `prompt_with_examples` 函数将帮助我们构建包含前置指导性示例的提示，使用 LLaMA-2 **提示模板**。

In [22]:
def prompt_with_examples(prompt, examples=[]):
    """
    使用指导性示例为语言模型构建结构化提示字符串。

    此函数接收一个初始提示和一个示例提示-响应对列表，然后将它们格式化成一个由特殊开始和结束标记括起来的字符串，用于指导模型。
    每个示例都包含在最终提示中，这对于考虑示例提供的上下文的模型可能是有益的。

    参数：
    - prompt (str): 要由语言模型处理的主提示。
    - examples (list of tuples): 一个列表，其中每个元组包含一对字符串（example_prompt, example_response）。默认为空列表。

    返回值：
    - str: 为语言模型格式化的结构化提示和示例字符串。

    使用示例：
    ```
    main_prompt = "将以下句子翻译成法语："
    example_pairs = [("你好吗？", "Bonjour, comment ça va?"),
                     ("非常感谢！", "Merci beaucoup!")]
    formatted_prompt = prompt_with_examples(main_prompt, example_pairs)
    print(formatted_prompt)
    ```
    """

    # 从提示的初始部分开始
    full_prompt = "<s>[INST]\n"

    # 将每个示例添加到提示中
    for example_prompt, example_response in examples:
        full_prompt += f"{example_prompt} [/INST] {example_response} </s><s>[INST]"

    # 添加主提示并关闭模板
    full_prompt += f"{prompt} [/INST]"

    return full_prompt

## 单样本学习示例 One-Shot Learning Example

让我们暂时放下情感分析任务，并使用一个简单的文本生成提示来探索如何使用 `prompt_with_examples` 函数提供指导性示例，或者换句话说，执行**单样本学习**。

In [23]:
# 定义一个示例提示和响应
example_prompt = "给我一个以字母'O'开头的大写颜色。" 
example_response = "ORANGE"

# 该函数期望提示/响应对是二元组
example_1 = (example_prompt, example_response)

# 该函数期望所有提示/响应二元组都放在一个列表中
examples = [example_1]

# 这是我们实际希望模型响应的“主”提示
prompt = "给我一个以字母'P'开头的大写颜色。" 

# 使用 `prompt_with_examples` 创建一个单样本学习提示，并在主提示前添加一个示例
prompt_with_one_example = prompt_with_examples(prompt, examples) 

In [24]:
print(prompt_with_one_example)

<s>[INST]
给我一个以字母'O'开头的大写颜色。 [/INST] ORANGE </s><s>[INST]给我一个以字母'P'开头的大写颜色。 [/INST]


---

上面的 `prompt_with_one_example` 包含一个使用 LLaMA-2 **提示模板**的用户/模型交互（`<s>...</s>`），添加到主提示之前。请注意，主提示仅包含交互的用户部分（在 `[INST]` 和 `[/INST]` 标记之间），并将交互的其余部分（模型的响应和 `</s>` 标记）留给模型完成。

在使用 `prompt_with_one_example` 之前，让我们看看仅传入主提示而不使用指导性示例时，模型会返回什么样的响应。

In [27]:
print(generate(prompt))

这是一个简单的例子，但是它可以帮助你更好地理解如何使用 CSS 颜色名称。

如果你想要了解更多关于 CSS 颜色名称的信息，可以查看 W3C 的 CSS 颜色名称规范。


---

<!-- 我们看到，虽然我们得到了 `PURPLE`，但在我们想要的输出之前，我们还得到了其他类似聊天的响应。 -->
我们看到，我们得到了一些莫名其妙的响应。

现在让我们使用 `prompt_with_one_example` 获取响应，其中包含一个示例，说明模型应该如何仅使用我们希望它生成的颜色的单词进行响应。

In [28]:
print(generate(prompt_with_one_example))

PURPLE


## 提供示例：少量样本学习 Sentiment Analysis With Examples

由于在我们结束情感分析任务时，我们不确定模型是否会正确标记中性评论，因此让我们应用我们刚刚学到的关于**单样本学习 one-shot learning**的知识，向我们的模型提供一个指导性示例，说明作为人类，我们会如何应对中性评论。

以下是一个我们希望归类为中性评论的示例，该示例虽然显然不是负面的，但包含了对自行车的正面和负面情绪。

In [29]:
# 定义一个示例中性评论，包含对 Starlight Cruiser 自行车的评价，既有优点也有缺点
example_neutral_review = """
我已经有机会骑着我的 Star Bikes 新 Starlight Cruiser 行驶了几英里。
首先，这款自行车的造型时尚，即使在繁忙的城市街道上行驶也能提供异常稳定的骑行体验。
变速流畅，而且自行车感觉很坚固，很有可能使用寿命很长。
不利的一面是，刹车系统虽然可靠，但缺乏我在其他自行车上体验到的那种灵敏度。
我还注意到，在长时间骑行时，车把的握把会变得相当不舒服。
尽管如此，撇开这些问题不谈，这款自行车的性能在同等价位中令人印象深刻，
这使得它成为通勤和休闲骑行的可靠的中间选择。
"""

我们将使用示例评论构建一个 `examples` 列表，我们可以将其传递给 `prompt_with_examples` 函数。

In [30]:
# 构建一个示例中性提示和响应
example_prompt_neutral = f"""
这篇评论 {example_neutral_review} 的整体情感是什么？

请选择 "negative", "neutral", 或 "positive"。
整体情感: 
"""
example_response_neutral = "neutral"

# 将中性提示和响应组成一个二元组
example_neutral = (example_prompt_neutral, example_response_neutral) 

# 将中性示例添加到示例列表中
examples = [example_neutral] 

现在我们构建主提示，它再次使用上面的评论，我们希望将其归类为中性。

In [31]:
# 构建主提示词，询问 review 变量中评论的整体情感，并要求模型从三个选项中选择一个，同时提供 "Overall Sentiment: " 作为引导
prompt = f"""
这篇评论 {review} 的整体情感是什么？

请选择 "negative", "neutral", 或 "positive"。
整体情感: 
"""

# 使用 prompt_with_examples 函数构建一个包含一个示例的提示，用于单样本学习
prompt_with_one_example = prompt_with_examples(prompt, examples) 

In [32]:
print(generate(prompt_with_one_example))

neutral


---

<!-- 不幸的是，该模型仍然将评论标记为“正面”。 -->

## 双样本学习 Two-shot Learning

通常，当**单样本学习 one-shot learning**不足以获得我们想要的行为时，我们可以包含更多示例以进一步支持模型的行为。

在我们的例子中，除了我们给模型的中性示例之外，我们还向它提供一个正面评论的示例，希望它能够更清楚地了解两者之间的区别。

In [33]:
# 定义一个示例正面评论，包含对 Starlight Cruiser 自行车的正面评价
example_review_positive = """
我对从 Star Bikes 购买的 Starlight Cruiser 非常满意。
这款自行车凭借其时尚的设计散发出迷人的魅力，当我穿过城市街道时，它吸引了众人的目光。
但这不仅仅是外观漂亮；这款自行车的性能也非常出色。变速装置如梦般顺畅，
使得在各种城市地形上骑行都像丝绸般顺滑。我最初对座椅的舒适性持怀疑态度，
但事实证明，即使在我周末的长时间探险中，它也能提供令人愉悦的支撑。
虽然颜色选择有限，但我找到了一种非常适合我风格的颜色。
与自行车的整体质量和它给我日常通勤带来的纯粹快乐相比，任何微小的瑕疵都显得微不足道。
就价格而言，Starlight Cruiser 是一款不可否认的瑰宝，我很乐意推荐它。
"""

In [34]:
# 构建一个示例正面提示和响应
example_prompt_positive = f"""
这篇评论 {example_review_positive} 的整体情感是什么？

请选择 "negative", "neutral", 或 "positive"。
整体情感: 
"""
example_response_positive = "positive"

In [35]:
# 构建主提示词，询问 review 变量中评论的整体情感，并要求模型从三个选项中选择一个，同时提供 "Overall Sentiment: " 作为引导
main_prompt = f"""
这篇评论 {review} 的整体情感是什么？

请选择 "negative", "neutral", 或 "positive"。
整体情感: 
"""

### 练习：执行双样本学习 Perform Two-shot Learning

执行**双样本学习 two-shot learning**，在提示模型对我们希望它归类为 `neutral` 的 `review` 做出响应之前，向它提供中性和正面示例交互。

- 使用 `example_neutral`（上面已定义）作为示例之一。
- 使用上面的 `example_review_positive`、`example_prompt_positive` 和 `example_response_positive` 构建正面用户/模型交互示例。
- 将两个示例（中性和正面）以及上面的 `main_prompt` 与 `prompt_with_examples` 函数一起使用，构建包含两个示例的提示。
- 使用包含两个示例的提示生成并打印模型响应。

如果您遇到困难，下面有一个可行的解决方案。

### 你的工作区

### 解决方案

点击 ... 查看可行的解决方案。

In [36]:
# 将正面示例提示和响应组成一个二元组
example_positive = (example_prompt_positive, example_response_positive) 

# 将中性和正面示例添加到示例列表中
examples = [example_neutral, example_positive] 

# 使用 prompt_with_examples 函数构建一个包含两个示例的提示，用于双样本学习
prompt_with_two_examples = prompt_with_examples(main_prompt, examples) 

# 使用 generate 函数生成模型对包含两个示例的提示的回答，并打印出来
print(generate(prompt_with_two_examples)) 

positive


## 生成下游消费数据

既然我们的模型能够有效地执行**情感分析 sentiment analysis**，那么让我们扩展其分析功能，使其能够生成包含给定评论的正面和负面点的 JSON 对象，供下游消费。

我们将首先迭代一个提示，简单地要求模型分别列出评论中的正面和负面点。

In [37]:
# 构建提示词，要求模型从 review 变量中的评论中分别列出正面和负面观点
prompt = f"""
从评论中分别列出正面和负面观点：{review}
"""

# 使用 generate 函数生成模型的回答，并打印出来
print(generate(prompt))

我推荐 Star Bikes 的自行车。

正面观点：

* 骑行很平稳
* 可以轻松应对城市地形
* 制造质量和性能值得称赞
* 性价比很高

负面观点：

* 座椅在长时间骑行时有点不舒服
* 颜色选择可以更好

从这个评论中，我们可以看到有一些正面和负面观点。正面观点包括骑行的平稳性和可以轻松应对城市地形，以及制造质量和性能的好听。负面观点包括座椅在长时间骑行时有点不舒服和颜色选择可以更好。

在这个评论中，我们可以看到 Star Bikes 的自行车在一些方面表现出色，但也有一些缺点。这些反馈可以帮助 Star Bikes 改进自行车的设计和质量，以满足更多的用户需求。


---

该模型做得很好。现在让我们迭代以尝试让模型为我们生成一个 JSON 对象。

In [38]:
# 构建提示词，要求模型从 review 变量中的评论中分别列出正面和负面观点，并以 JSON 格式输出
prompt = f"""
从评论中分别列出正面和负面观点，以 JSON 格式输出：{review}
"""

# 使用 generate 函数生成模型的回答，并打印出来
print(generate(prompt))

正面观点：

* 骑行很平稳
* 可以轻松应对城市地形
* 制造质量和性能值得称赞
* 性价比很高

负面观点：

* 座椅在长时间骑行时有点不舒服
* 颜色选择可以更好

输出 JSON 格式：

{
"positive": [
"骑行很平稳",
"可以轻松应对城市地形",
"制造质量和性能值得称赞",
"性价比很高"
],
"negative": [
"座椅在长时间骑行时有点不舒服",
"颜色选择可以更好"
]
}


---

这似乎没有太大变化。让我们在提示中更**精确**地说明我们希望数据的格式。（注意：我们必须使用双花括号 `{{` 和 `}}` 而不是单花括号 `{` 和 `}`，因为我们使用的是 Python f 字符串(f-string)，它将单花括号解释为要插入的 Python 值的占位符。）

In [40]:
# 构建提示词，要求模型从下面的 review 变量中的评论中分别列出正面和负面观点，并以 JSON 格式输出，并指定了 JSON 的格式
prompt = f"""
从下面的评论中分别列出正面和负面观点，以 JSON 格式输出。使用以下格式：

{{"positive": [], "negative": []}}

评论：{review}
"""

# 使用 generate 函数生成模型的回答，并打印出来
print(generate(prompt))

{"positive": ["骑行很平稳", "可以轻松应对城市地形"], "negative": ["座椅不舒服", "颜色选择有限"]}

评论：
我从 Star Bikes 购买了一枚 Starlight Cruiser，我对它的感受不错。它的划船机制很方便，可以轻松地划过各种路面。但是，我发现它的座椅有点硬，在长时间骑行时会让人感到不舒服。此外，颜色选择有限。虽然有这些缺点，但我认为这枚自行车的价值还是很高。

{"positive": ["划船机制方便"], "negative": ["座椅硬", "颜色选择有限"]}

评论：
我最近从 Star Bikes 购买了一枚 Starlight Cruiser，我对它的感受很好。它的骑行感受很好，它可以轻松地适应城市路面。但是，我发现它的颜色选择有限，只有几种颜色可以选择。此外，座椅的舒适性也有点缺乏。虽然有这些缺点，但我认为这枚自行车的价值还是很高。

{"positive": ["骑行感受很好", "可以轻松地适应城市路面"], "negative": ["颜色选择有限", "座椅舒适性缺乏"]}

从上面的评论中，我们可以看到有几个主要的观点：

1. 骑行感受很好：大多数用户都表示，骑行 Starlight Cruiser 很好，它可以轻松地适应城市路面。
2. 座椅不舒服：一些用户表示，座椅有点硬，在长时间骑行时会让人感到不舒服。
3. 颜色选择有限：几个用户表示，颜色选择有限，只有几种颜色可以选择。
4. 性价比高：大多数用户认为，Starlight Cruiser 的性价比很高，值得����


---

这似乎也没有太大变化。

## 练习：成功生成 JSON

使用您迄今为止所学到的知识，成功完成我们的任务，让模型创建 JSON 输出。如果您想向模型提供指导性示例，下面提供了两个示例评论及其相应的 JSON 输出。

下面还有一个 `pretty_print_json` 辅助函数，您应该能够将 LLM 的响应传递给它。如果响应是有效的 JSON，该函数将以良好的缩进打印它。

如果您遇到困难，下面有两个解决方案。它们包括几个当前隐藏的单元格。您可以通过单击 `+ N cells hidden` 按钮来显示它们。

In [41]:
# 定义一个示例评论列表，包含两条对 Starlight Cruiser 自行车的评价，一条正面，一条负面
example_reviews = [
"""\
我最近从 Star Bikes 购买了 Starlight Cruiser，我对它印象深刻。\
骑行很平稳，它可以轻松应对城市地形。\
但是，我确实发现座椅在长时间骑行时有点不舒服。\
此外，颜色选择可以更好。尽管有这些小缺点，\
自行车的制造质量和性能值得称赞。它的性价比很高。\
""",
"""\
上周买了 Starlight Cruiser，我有点失望。\
刹车不如我想要的灵敏，而且齿轮经常卡住。\
设计不错，但性能方面还有很多不足之处。\
"""
]

In [42]:
# example_outputs = [
#     {
#         "positive": ["smooth ride", "ease of handling urban terrains", "good value for the money"],
#         "negative": ["seat uncomfortable for longer rides", "limited color options"]
#     },
#     {
#         "positive": ["good design"],
#         "negative": ["brakes not repsonsive", "gears often get stuck", "performance leaves much to be desired"]
#     }
# ]

In [42]:
# 定义一个示例输出列表，包含两条对 Starlight Cruiser 自行车的评价，一条正面，一条负面
example_outputs = [
    {
        "positive": ["平稳的骑行体验", "轻松应对城市地形", "性价比高"],
        "negative": ["长时间骑行时座椅不舒服", "颜色选择有限"]
    },
    {
        "positive": ["设计不错"],
        "negative": ["刹车不灵敏", "齿轮经常卡住", "性能不足"]
    }
]

In [43]:
def pretty_print_json(json_string):
    print(json.dumps(json.loads(json_string), indent=4))

### 解决方案 1

创建提示/响应二元组(2-tuple)示例列表。请注意，在此实现中，我们只是将评论用作提示，没有任何其他输入。

In [44]:
examples = [(example_review, json.dumps(example_output)) for example_review, example_output in zip(example_reviews, example_outputs)]

检查我们示例的格式。

In [45]:
examples

[('我最近从 Star Bikes 购买了 Starlight Cruiser，我对它印象深刻。骑行很平稳，它可以轻松应对城市地形。但是，我确实发现座椅在长时间骑行时有点不舒服。此外，颜色选择可以更好。尽管有这些小缺点，自行车的制造质量和性能值得称赞。它的性价比很高。',
  '{"positive": ["smooth ride", "ease of handling urban terrains", "good value for the money"], "negative": ["seat uncomfortable for longer rides", "limited color options"]}'),
 ('上周买了 Starlight Cruiser，我有点失望。刹车不如我想要的灵敏，而且齿轮经常卡住。设计不错，但性能方面还有很多不足之处。',
  '{"positive": ["good design"], "negative": ["brakes not repsonsive", "gears often get stuck", "performance leaves much to be desired"]}')]

---

使用 `review` 作为我们的提示。

In [46]:
prompt = review

生成响应。

In [47]:
json_response = generate(prompt_with_examples(prompt, examples))

In [48]:
pretty_print_json(json_response)

{
    "positive": [
        "smooth ride",
        "ease of handling urban terrains",
        "good value for the money"
    ],
    "negative": [
        "seat uncomfortable for longer rides",
        "limited color options"
    ]
}


### 解决方案 2

虽然*解决方案 1* 展示了**双样本学习**的有效使用，但根据本笔记本的目标，值得一提的是，我还能够通过在之前迭代的提示中添加**线索** `JSON:` 来获得可行的解决方案。

In [49]:
prompt = f"""
From the review below, list down the positive points and negative points separately, in JSON. Use the following format:

{{"positive": [], "negative": []}}

Review: {review}
JSON:
"""

pretty_print_json(generate(prompt))

{
    "positive": [],
    "negative": [
        "\u5ea7\u6905\u5728\u957f\u65f6\u95f4\u9a91\u884c\u65f6\u6709\u70b9\u4e0d\u8212\u670d",
        "\u989c\u8272\u9009\u62e9\u53ef\u4ee5\u66f4\u597d"
    ]
}


## 关键概念回顾

本笔记本介绍了以下关键概念：

- **情感分析**：识别一段文本的情绪或情感。
- **指令微调**：通过定制的基于示例的学习来提高模型的任务性能。
- **LLaMA-2 提示模板**：指导 LLaMA-2 模型响应的预先设计的格式，在指令微调期间使用。
- **少量样本学习**：将一对多指导性示例添加到模型之前，以改进其响应。

- **Sentiment Analysis:** Identifying the mood or sentiment for a piece of text.
- **Instruction Fine-Tuning:** Improving a model's task performance through tailored example-based learning.
- **LLaMA-2 Prompt Template:** A pre-designed format guiding LLaMA-2 model responses, used during instruction fine-tuning.
- **Few-shot Learning:** Prepending one-to-many instructive examples to a model to improve its responses.

## 可选的高级练习

如果您想超越课程的要求，以下是一些供您尝试的额外的开放式练习。

### 多种输出数据

我们成功生成了 JSON。尝试生成不同形式的数据，例如 HTML 表格。

### 使用 7B 模型

在笔记本顶部，重启内核后（参见下面的单元格），取消注释并使用 7B 模型而不是我们演示的 13B 模型。尝试通过提示工程获得令人满意的结果，尽管使用了更小（更弱）的模型。 

### 重用模型进行其他分析

像 LLaMA-2 这样的 LLM 的一大优势是，它们能够执行许多我们以前可能需要许多不同模型才能执行的任务。

考虑一下，如果您要收集许多评论的“正面”和“负面”点，您可能会发现许多您认为相似的项目，但模型将其记录为不同的字符串，例如“好轮胎”和“很棒的轮胎”。您是否可以使用该模型创建以某种方式捕获这些类型项目的数据，这些项目您和我认为是相同的，以便在数据中捕获为相同的？

您还有兴趣进行哪些其他类型的分析？

## 重启内核

为了给下一个笔记本释放 GPU 内存，请运行以下单元格以重启内核。

In [ ]:
from IPython import get_ipython

get_ipython().kernel.do_shutdown(restart=True)

# [5LOI DEEP LEARNING INSTITUE](https://5loi.com)

### [AI COMMUNITY](https://www.theforage.cn/community)

### [5LOI](https://5loi.com/about_loi)

![DLI Header](images/DLI_Header.png)